In [3]:
my_clique_list=[51, 61, 107, 176, 185, 220, 251, 293, 339, 349, 405, 434, 471, 478, 567, 574, 603, 636, 692, 698, 748, 786, 821, 848, 857, 898, 980, 986]

In [4]:
import pickle
import os

def validate_and_report_clique(clique_indices, dimension=5):
    """
    Validates a clique and saves a detailed neighbor report to a file.
    Header includes the full list of vertices in the clique.
    """
    # 1. Load the adjacency data
    filename = f'keller_codes_d{dimension}.data'
    try:
        with open(filename, 'rb') as f:
            codes_int = pickle.load(f)
    except FileNotFoundError:
        print(f"❌ Error: {filename} not found.")
        return

    n = len(codes_int)
    clique_indices = sorted(clique_indices)
    clique_set = set(clique_indices)
    
    # Adjacency check helper (Bitwise NAM logic)
    def are_neighbors(u, v):
        a, b = (u, v) if u < v else (v, u)
        return ((codes_int[a] >> b) & 1) == 0

    # 2. Decoding function (Base-4 coordinates)
    def decode_vertex(idx):
        coords = []
        temp = idx
        for _ in range(dimension):
            coords.append(temp % 4)
            temp //= 4
        return coords[::-1] 

    # 3. Validation Logic
    conflicts = []
    for i in range(len(clique_indices)):
        for j in range(i + 1, len(clique_indices)):
            u, v = clique_indices[i], clique_indices[j]
            if not are_neighbors(u, v):
                conflicts.append((u, v))
    is_valid = len(conflicts) == 0

    # 4. Generate the Report
    output_fn = f"clique_validation_report_d{dimension}.txt"
    with open(output_fn, "w", encoding="utf-8") as f:
        f.write("="*70 + "\n")
        f.write(f"KELLER-{dimension} CLIQUE VALIDATION REPORT\n")
        f.write("="*70 + "\n")
        f.write(f"Status:        {'✅ VALID (Perfect Clique)' if is_valid else '❌ INVALID'}\n")
        f.write(f"Clique Size:   {len(clique_indices)}\n")
        
        # ADDED: Full list of vertices in the header
        f.write(f"Full Clique:   {clique_indices}\n")
        
        if not is_valid:
            f.write(f"Conflicts:     {len(conflicts)} pair(s) fail adjacency\n")
        f.write("="*70 + "\n\n")

        for v in clique_indices:
            coords = decode_vertex(v)
            # Find all neighbors in the entire graph
            neighbors = [u for u in range(n) if u != v and are_neighbors(v, u)]
            
            # Count how many of this vertex's neighbors are in the clique
            clique_neighbors = [nb for nb in neighbors if nb in clique_set]
            
            f.write(f"Vertex {v:4} | Coordinates: {coords}\n")
            f.write(f"Internal Adjacency: {len(clique_neighbors)}/{len(clique_indices)-1} clique members\n")
            f.write(f"Total Graph Neighbors: {len(neighbors)}\n")
            f.write("Neighbors List (* = clique member):\n")
            
            formatted_neighbors = []
            for nb in sorted(neighbors):
                if nb in clique_set:
                    formatted_neighbors.append(f"{nb}*")
                else:
                    formatted_neighbors.append(str(nb))
            
            # Write neighbor list with line wrapping for better text file viewing
            line_width = 15
            for i in range(0, len(formatted_neighbors), line_width):
                f.write(", ".join(formatted_neighbors[i:i+line_width]) + ",\n")
            
            f.write("-" * 50 + "\n")

    print(f"✅ Detailed report generated: {output_fn}")
    if not is_valid:
        print(f"⚠️ Warning: Found {len(conflicts)} conflicts.")

# Usage: 
validate_and_report_clique(my_clique_list)

✅ Detailed report generated: clique_validation_report_d5.txt
